In [0]:
# In this demo I am joining stream and a datasets
# So we should have one dataset (static) data in 'dbfs:/FileStore/datasets/movies'

# And the other data(stream data) will be in Azure Eventhub

##### TODO Recording

- Go to the event hub namespace that we have already created
- loony-eventhub-namespace

- Create eventhub
- Click on +Event hub
- Add following setails

	name : loony-ratings

- Click on create
- Event hub has created in the namespace
- We can see the eventhub at the bottom in the namespace page

- We need two details from this page now
	name : loony-ratings
	primary key

- To get the primary key go to 'Shared access policies' which is there in the left side of the namespace page
- Then click on the 'RootManageSharedAccessKey' 
- Then the right side keys will be active
- Copy the connection string primary key

	Connection string–primary key:
	Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0=

- Go to the Databricks workspace
- Have StreamingStaticJoin open on one tab
- Have RatingsSource open on another tab

##### Dataset:
https://www.kaggle.com/danielgrijalvas/movies

In [0]:
dbutils.fs.mkdirs("dbfs:/FileStore/datasets/movies")

Out[2]: True

In [0]:
import json

from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

##### Static Data

In [0]:
static_schema = StructType([StructField("name", StringType(), True),
                            StructField("year", IntegerType(), True),
                            StructField("director", StringType(), True),
                            StructField("writer", StringType(), True),
                            StructField("star", StringType(), True)
                           ])

In [0]:
static_data = spark.read\
                   .format("csv")\
                   .option("header", "true")\
                   .schema(static_schema )\
                   .load("dbfs:/FileStore/datasets/movies/")

static_data.display()

name,year,director,writer,star
Stir Crazy,1980,Sidney Poitier,Bruce Jay Friedman,Gene Wilder
Cruising,1980,William Friedkin,William Friedkin,Al Pacino
Heaven's Gate,1980,Michael Cimino,Michael Cimino,Kris Kristofferson
The Final Countdown,1980,Don Taylor,Thomas Hunter,Kirk Douglas
Xanadu,1980,Robert Greenwald,Richard Christian Danus,Olivia Newton-John
Urban Cowboy,1980,James Bridges,Aaron Latham,John Travolta
Altered States,1980,Ken Russell,Paddy Chayefsky,William Hurt
Little Darlings,1980,Ron Maxwell,Kimi Peck,Tatum O'Neal
Raise the Titanic,1980,Jerry Jameson,Adam Kennedy,Jason Robards
Brubaker,1980,Stuart Rosenberg,W.D. Richter,Robert Redford


##### Streaming Data

In [0]:
primaryKey = "Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0="
entityPath = "EntityPath=loony-ratings"

connectionString = primaryKey + ";" + entityPath

ehConf = {}

startOffset = "-1"

startingEventPosition = {
  "offset": startOffset,  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)
ehConf["eventhubs.startingPosition"] = json.dumps(startingEventPosition)

In [0]:
streaming_schema = StructType([StructField("name", StringType(), True),
                               StructField("rating", StringType(), True),
                               StructField("score", FloatType(), True)])

##### TODO Recording

- Switch to the RatingsSource notebook
- Run the cells
- Pass in the following data

        event_data_batch.add(EventData('{ "name": "The Shining", "rating": "R", "score": 8.4 }'))
        event_data_batch.add(EventData('{ "name": "The Blue Lagoon", "rating": "R", "score": 5.8}'))
        event_data_batch.add(EventData('{ "name": "Star Wars: Episode V - The Empire Strikes Back", "rating": "PG", "score": 8.7}'))

In [0]:
streaming_data = spark.readStream \
                      .format("eventhubs") \
                      .options(**ehConf) \
                      .load()

In [0]:
streaming_data = streaming_data.selectExpr("CAST(body AS STRING)")

streaming_data.display()

body
"{ ""name"": ""The Shining"", ""rating"": ""R"", ""score"": 8.4 }"
"{ ""name"": ""The Blue Lagoon"", ""rating"": ""R"", ""score"": 5.8}"
"{ ""name"": ""Star Wars: Episode V - The Empire Strikes Back"", ""rating"": ""PG"", ""score"": 8.7}"
"{ ""name"": ""Airplane!"",""rating"": ""PG"", ""score"": 7.7 }"
"{ ""name"": ""Caddyshack"", ""rating"": ""R"", ""score"": 7.3 }"
"{ ""name"": ""Friday the 13th"", ""rating"": ""R"", ""score"": 6.4}"
"{ ""name"": ""The Blues Brothers"", ""rating"": ""R"", ""score"": 7.9}"
"{ ""name"": ""Raging Bull"", ""rating"": ""R"", ""score"": 8.2 }"
"{ ""name"": ""Lagaan"", ""rating"": ""PG"", ""score"": 9.2 }"
"{ ""name"": ""Star Wars: Episode V - The Empire Strikes Back"", ""rating"": ""PG"", ""score"": 8.2}"


In [0]:
streaming_data = streaming_data.select(from_json(col("body").cast("string"), streaming_schema)) \
                          .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                          .select(col('data.*'))

streaming_data.display()

name,rating,score
The Shining,R,8.4
The Blue Lagoon,R,5.8
Star Wars: Episode V - The Empire Strikes Back,PG,8.7
Airplane!,PG,7.7
Caddyshack,R,7.3
Friday the 13th,R,6.4
The Blues Brothers,R,7.9
Raging Bull,R,8.2
Lagaan,PG,9.2
Star Wars: Episode V - The Empire Strikes Back,PG,8.2


##### Outer join

In [0]:
# Join all elements of the batch with all elements of the stream

In [0]:
outer_join = static_data.join(streaming_data)

outer_join.display()

name,year,director,writer,star,name,rating,score
Stir Crazy,1980,Sidney Poitier,Bruce Jay Friedman,Gene Wilder,The Shining,R,8.4
Cruising,1980,William Friedkin,William Friedkin,Al Pacino,The Shining,R,8.4
Heaven's Gate,1980,Michael Cimino,Michael Cimino,Kris Kristofferson,The Shining,R,8.4
The Final Countdown,1980,Don Taylor,Thomas Hunter,Kirk Douglas,The Shining,R,8.4
Xanadu,1980,Robert Greenwald,Richard Christian Danus,Olivia Newton-John,The Shining,R,8.4
Urban Cowboy,1980,James Bridges,Aaron Latham,John Travolta,The Shining,R,8.4
Altered States,1980,Ken Russell,Paddy Chayefsky,William Hurt,The Shining,R,8.4
Little Darlings,1980,Ron Maxwell,Kimi Peck,Tatum O'Neal,The Shining,R,8.4
Raise the Titanic,1980,Jerry Jameson,Adam Kennedy,Jason Robards,The Shining,R,8.4
Brubaker,1980,Stuart Rosenberg,W.D. Richter,Robert Redford,The Shining,R,8.4


In [0]:
# Let's add with what name we have to merge the data
# It showed all the join which has both data values

In [0]:
right_outer_join = static_data.join(streaming_data, on=["name"], how="right_outer")

right_outer_join.display()

name,year,director,writer,star,rating,score
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson,R,8.4
The Blue Lagoon,1980,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,R,5.8
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.7
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays,PG,7.7
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase,R,7.3
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,R,6.4
The Blues Brothers,1980,John Landis,Dan Aykroyd,John Belushi,R,7.9
Raging Bull,1980,Martin Scorsese,Jake LaMotta,Robert De Niro,R,8.2
Lagaan,null,null,null,null,PG,9.2
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2


#### TODO Recording

- Both the cells below should throw errors

In [0]:
left_outer_join_rightstream = static_data.join(streaming_data, on=["name"], how="left_outer")

left_outer_join_rightstream.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3541484190323439> in <module> 
 1 left_outer_join_rightstream = static_data . join ( streaming_data , on = [ "name" ] , how = "left_outer" ) 
 2 
 ----> 3 left_outer_join_rightstream . display ( ) 

 /databricks/python_shell/dbruntime/monkey_patches.py in df_display (df, *args, **kwargs) 
 27 df . display ( ) is an alias for display ( df ) . Run help ( display ) for more information . 
 28 """
 ---> 29 display ( df , * args , ** kwargs ) 
 30 
 31 @ when_imported ( 'pyspark.sql' ) 

 /databricks/python_shell/scripts/PythonShellImpl.py in display (self, input, *args, **kwargs) 
 1120 # exists and then if it does, whether it is actually streaming. 
 1121 if hasattr ( input , 'isStreaming' ) and input . isStreaming : 
 -> 1122 handleStreamingDataFrame ( input , self . sc , self . sqlContext , self . entry_point , kwargs ) 
 1123 else : 
 1124 if kwargs . get ( 'streamName' ) : 

 /databricks/python_shell/dbruntime/display.py in handleStreamingDataFrame (input, sc, sqlContext, entry_point, kwargs) 
 109 .DisplayHelper.getStreamName())
 110 
 --> 111 entry_point.getDriverSparkHooks().displayStreamingDataFrame(input._jdf, name, trigger,
 112 kwargs.get('checkpointLocation'))
 113 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : LeftOuter join with a streaming DataFrame/Dataset on the right and a static DataFrame/Dataset on the left is not supported;
Join LeftOuter, (name#457 = name#1054)
:- Relation[name#457,year#458,director#459,writer#460,star#461] csv
+- Project [data#1052.name AS name#1054, data#1052.rating AS rating#1055, data#1052.score AS score#1056]
 +- Project [from_json(CAST(body AS STRING))#1050 AS data#1052]
 +- Project [from_json(StructField(name,StringType,true), StructField(rating,StringType,true), StructField(score,FloatType,true), cast(body#1004 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#1050]
 +- Project [cast(body#986 as string) AS body#1004]
 +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@c2cc379,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> P4w5ooxK264fEn4fSjKMh0CgCpv6wQLbtLTOzbqmvtyM/MK1uyEyPEhrqFUohdaoEq1ipYfu2zWSoFKQ6knyBLS/5gUDr3ADZyLcTE6KvSMk7p/pKjtZca2Dq+6rahexk0vGX03dVL6MMXNSFPbwoq890m5/e3/SJ67TeuugVvq2PEnOvg2JIssGDiJmDM+V2WTcgrC+wMZv9qyZjhhsR5bJ26rx9e/YBFu5uAaZhHJzcDsgMGkg3txM977iWWmDoOZZlNtnE6njf8sRVfKMJw==, eventhubs.startingPosition -> {"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}),None), eventhubs, [body#986, partition#987, offset#988, sequenceNumber#989L, enqueuedTime#990, publisher#991, partitionKey#992, properties#993, systemProperties#994]

In [0]:
right_outer_join_leftstream = streaming_data.join(static_data, on=["name"], how="right_outer")

right_outer_join_leftstream.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3541484190323376> in <module> 
 1 right_outer_join_leftstream = streaming_data . join ( static_data , on = [ "name" ] , how = "right_outer" ) 
 2 
 ----> 3 right_outer_join_leftstream . display ( ) 

 /databricks/python_shell/dbruntime/monkey_patches.py in df_display (df, *args, **kwargs) 
 27 df . display ( ) is an alias for display ( df ) . Run help ( display ) for more information . 
 28 """
 ---> 29 display ( df , * args , ** kwargs ) 
 30 
 31 @ when_imported ( 'pyspark.sql' ) 

 /databricks/python_shell/scripts/PythonShellImpl.py in display (self, input, *args, **kwargs) 
 1120 # exists and then if it does, whether it is actually streaming. 
 1121 if hasattr ( input , 'isStreaming' ) and input . isStreaming : 
 -> 1122 handleStreamingDataFrame ( input , self . sc , self . sqlContext , self . entry_point , kwargs ) 
 1123 else : 
 1124 if kwargs . get ( 'streamName' ) : 

 /databricks/python_shell/dbruntime/display.py in handleStreamingDataFrame (input, sc, sqlContext, entry_point, kwargs) 
 109 .DisplayHelper.getStreamName())
 110 
 --> 111 entry_point.getDriverSparkHooks().displayStreamingDataFrame(input._jdf, name, trigger,
 112 kwargs.get('checkpointLocation'))
 113 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : RightOuter join with a streaming DataFrame/Dataset on the left and a static DataFrame/DataSet on the right not supported;
Join RightOuter, (name#1054 = name#457)
:- Project [data#1052.name AS name#1054, data#1052.rating AS rating#1055, data#1052.score AS score#1056]
: +- Project [from_json(CAST(body AS STRING))#1050 AS data#1052]
: +- Project [from_json(StructField(name,StringType,true), StructField(rating,StringType,true), StructField(score,FloatType,true), cast(body#1004 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#1050]
: +- Project [cast(body#986 as string) AS body#1004]
: +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@c2cc379,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> P4w5ooxK264fEn4fSjKMh0CgCpv6wQLbtLTOzbqmvtyM/MK1uyEyPEhrqFUohdaoEq1ipYfu2zWSoFKQ6knyBLS/5gUDr3ADZyLcTE6KvSMk7p/pKjtZca2Dq+6rahexk0vGX03dVL6MMXNSFPbwoq890m5/e3/SJ67TeuugVvq2PEnOvg2JIssGDiJmDM+V2WTcgrC+wMZv9qyZjhhsR5bJ26rx9e/YBFu5uAaZhHJzcDsgMGkg3txM977iWWmDoOZZlNtnE6njf8sRVfKMJw==, eventhubs.startingPosition -> {"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}),None), eventhubs, [body#986, partition#987, offset#988, sequenceNumber#989L, enqueuedTime#990, publisher#991, partitionKey#992, properties#993, systemProperties#994]
+- Relation[name#457,year#458,director#459,writer#460,star#461] csv

###### TODO Recording

- Add more values to the eventhub stream

        event_data_batch.add(EventData('{ "name": "Lagaan", "rating": "PG", "score": 9.2 }'))
        event_data_batch.add(EventData('{ "name": "Andhadhun", "rating": "PG", "score": 8.4 }'))
        event_data_batch.add(EventData('{ "name": "Airplane!","rating": "PG", "score": 7.7 }'))
        event_data_batch.add(EventData('{ "name": "Caddyshack", "rating": "R", "score": 7.3 }'))
        event_data_batch.add(EventData('{ "name": "Friday the 13th", "rating": "R", "score": 6.4}'))
        event_data_batch.add(EventData('{ "name": "The Blues Brothers", "rating": "R", "score": 7.9}'))
        event_data_batch.add(EventData('{ "name": "Raging Bull", "rating": "R", "score": 8.2 }'))

- Scroll to the top and show the update of the outer_join and the right_outer_join. 
- Show that Lagaan has no match in the batch dataset

In [0]:
left_outer_join = streaming_data.join(static_data, on=["name"], how="left_outer")

left_outer_join.display()

name,rating,score,year,director,writer,star
The Shining,R,8.4,1980,Stanley Kubrick,Stephen King,Jack Nicholson
The Blue Lagoon,R,5.8,1980,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields
Star Wars: Episode V - The Empire Strikes Back,PG,8.7,1980,Irvin Kershner,Leigh Brackett,Mark Hamill
Airplane!,PG,7.7,1980,Jim Abrahams,Jim Abrahams,Robert Hays
Caddyshack,R,7.3,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase
Friday the 13th,R,6.4,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer
The Blues Brothers,R,7.9,1980,John Landis,Dan Aykroyd,John Belushi
Raging Bull,R,8.2,1980,Martin Scorsese,Jake LaMotta,Robert De Niro
Lagaan,PG,9.2,null,null,null,null
Star Wars: Episode V - The Empire Strikes Back,PG,8.2,1980,Irvin Kershner,Leigh Brackett,Mark Hamill


##### Inner join

###### TODO Recording

- Scroll and show that Lagaan and Andhadhun are not present in the result

In [0]:
inner_join = static_data.join(streaming_data, on=["name"], how="inner")

inner_join.display()

name,year,director,writer,star,rating,score
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson,R,8.4
The Blue Lagoon,1980,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,R,5.8
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.7
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays,PG,7.7
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase,R,7.3
Friday the 13th,1980,Sean S. Cunningham,Victor Miller,Betsy Palmer,R,6.4
The Blues Brothers,1980,John Landis,Dan Aykroyd,John Belushi,R,7.9
Raging Bull,1980,Martin Scorsese,Jake LaMotta,Robert De Niro,R,8.2
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2
Fame,1980,Alan Parker,Christopher Gore,Eddie Barth,R,6.6


In [0]:
selected_join = static_data.join(streaming_data, on=["name"], how="inner")\
                           .select("name", "director", "star", "score") 

selected_join.display()

name,director,star,score
The Shining,Stanley Kubrick,Jack Nicholson,8.4
The Blue Lagoon,Randal Kleiser,Brooke Shields,5.8
Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,8.7
Airplane!,Jim Abrahams,Robert Hays,7.7
Caddyshack,Harold Ramis,Chevy Chase,7.3
Friday the 13th,Sean S. Cunningham,Betsy Palmer,6.4
The Blues Brothers,John Landis,John Belushi,7.9
Raging Bull,Martin Scorsese,Robert De Niro,8.2
Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,8.2
Fame,Alan Parker,Eddie Barth,6.6


In [0]:
# Let's find the top ratings per movie now

In [0]:
top_scorers = selected_join.select("name", "score").where(selected_join.score > 8)

top_scorers.display()

name,score
The Shining,8.4
Star Wars: Episode V - The Empire Strikes Back,8.7
Raging Bull,8.2
Star Wars: Episode V - The Empire Strikes Back,8.2


In [0]:
inner_join_error = static_data.join(streaming_data, on=["star"], how="inner")

inner_join_error.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3541484190323388> in <module> 
 ----> 1 inner_join = static_data . join ( streaming_data , on = [ "star" ] , how = "inner" ) 
 2 
 3 inner_join . display ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in join (self, other, on, how) 
 1360 on = self . _jseq ( [ ] ) 
 1361 assert isinstance ( how , str ) , "how should be a string" 
 -> 1362 jdf = self . _jdf . join ( other . _jdf , on , how ) 
 1363 return DataFrame ( jdf , self . sql_ctx ) 
 1364 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : USING column `star` cannot be resolved on the right side of the join. The right-side columns: [name, rating, score]

##### TODO Recording

- Add more data to the eventhub

        event_data_batch.add(EventData('{ "name": "Star Wars: Episode V - The Empire Strikes Back", "rating": "PG", "score": 8.2}'))
        event_data_batch.add(EventData('{ "name": "Fame", "rating": "R", "score": 6.6}'))
        event_data_batch.add(EventData('{ "name": "Friday the 13th", "rating": "R", "score": 6.1}'))
        
- Scroll back up to the inner join and show the updates
- Records in the inner join go from 8 to 11
- top scorers will go from 3 to 4